# Quickstart: Compare runs, choose a model, and deploy it to a REST API

In this quickstart, you will:
- Run a hyperparameter sweep on a training script

- Compare the results of the runs in the MLFLOW UI

- Choose the best run and register it as a model

- Deploy the model to a REST API 

- Build a container image suitable for deployment to a cloud platform 

In [1]:
import keras
import pandas as pd
import numpy as np
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import mlflow
from mlflow.models import infer_signature



/Users/ashmijha/Downloads/My_Projects/MLflow/venv/lib/python3.13/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
## load the dataset
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [3]:
## Split the data into training and testing sets
train, test = train_test_split(data, test_size=0.25, random_state=42)

In [4]:
# Split the data into independent and dependent features
# Training set
train_x = train.drop(columns=["quality"], axis=1).values
train_y = train["quality"].values.ravel()

# Testing set
test_x = test.drop(columns=["quality"], axis=1).values
test_y = test["quality"].values.ravel()

# Split the training set into training and validation sets
train_x, val_x, train_y, val_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)

# Settingg the input-output signature
signature = infer_signature(train_x, train_y)

In [5]:
### ANN model 

def train_evaluate_model(params, epochs, train_x, train_y, val_x, val_y, test_x, test_y):
    # Calculate mean and variance for normalization layer
    mean = np.mean(train_y, axis=0)
    var = np.var(train_y, axis=0)
    # Build the model
    model = keras.Sequential(
        [
            keras.Input(shape=(train_x.shape[1],)),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1)
        ]
    )
    # Compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["learning_rate"],
        momentum=params["momentum"],
    ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()]
    )
    
    ## Train the ANN Model with the given hyperparameters
    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=epochs, batch_size=64)

        ## Evaluate the model 
        eval_result = model.evaluate(val_x, val_y, batch_size=64)

        eval_rmse = eval_result[1]

        ## Log the params and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        ## Log the model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}


In [6]:
def objective(params):
    #MLFLOW will track the parameters and result for each run
    return train_evaluate_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        val_x=val_x,
        val_y=val_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [7]:
space ={
    "learning_rate": hp.loguniform("learning_rate", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}

In [10]:
# Set the mlflow experiment name
mlflow.set_experiment("DL_Hyperopt_ANN_Wine_Quality")

#start mlflow run
with mlflow.start_run():
# Hyperparameter optimization using hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials,
    )

    #fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    #log the best hyperparameters, loss and metric 
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    #Print the best hyperparameters and loss
    print("Best parameters:", best)
    print("Best eval rmse:", best_run["loss"])

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 6s 139ms/step - loss: 34.1411 - root_mean_squared_error: 5.8430
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 8.4758 - root_mean_squared_error: 2.9113 - val_loss: 7.5002 - val_root_mean_squared_error: 2.7386

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 8.2075 - root_mean_squared_error: 2.8649
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - loss: 6.6898 - root_mean_squared_error: 2.5865 - val_loss: 6.9181 - val_root_mean_squared_error: 2.6302

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 6.0704 - root_mean_squared_error: 2.4638
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 707us/step - loss: 6.0207 - root_mean_squared_error: 2.4537 - val_loss: 6.1602 - val_root_mean_squared_error: 2.4820

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 4.9519 - root_mean_squared_error: 2.2253
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 869us/st

2025/12/09 20:06:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 6s 138ms/step - loss: 1154.0017 - root_mean_squared_error: 33.9706
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 77319944.0000 - root_mean_squared_error: 8793.1758 - val_loss: 910742.8125 - val_root_mean_squared_error: 954.3285

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 910432.0000 - root_mean_squared_error: 954.1656
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - loss: 412891.8438 - root_mean_squared_error: 642.5666 - val_loss: 123727.5547 - val_root_mean_squared_error: 351.7493

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 123667.9688 - root_mean_squared_error: 351.6646
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 730us/step - loss: 53646.8008 - root_mean_squared_error: 231.6178 - val_loss: 15757.4189 - val_root_mean_squared_error

2025/12/09 20:06:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



...

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 6s 146ms/step - loss: 59.4541 - root_mean_squared_error: 7.7107
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2182149050600521728.0000 - root_mean_squared_error: 1477209856.0000 - val_loss: 789282488320.0000 - val_root_mean_squared_error: 888415.6875

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 789282422784.0000 - root_mean_squared_error: 888415.6875
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 784us/step - loss: 28648282112.0000 - root_mean_squared_error: 169258.0312 - val_loss: 6.7597 - val_root_mean_squared_error: 2.5999

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 7.4404 - root_mean_squared_error: 2.7277
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 765us/step - loss: 1.1017 - root_mean_squared_error: 1.0496 - val_loss: 0.7823 - val_root_me

2025/12/09 20:07:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 4/4 [00:17<00:00,  4.26s/trial, best loss: 0.8844864964485168]

2025/12/09 20:07:10 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Best parameters: {'learning_rate': np.float64(0.0540598172037221), 'momentum': np.float64(0.5554987688335554)}
Best eval rmse: 0.8844864964485168
